In [ ]:
import numpy as np
import pandas as pd

'mandarino.ant@gmail.com'

In [109]:
from density_matrix_from_exp import generate_rho_rho_delta
from deprecated.density_matrix_gen import prova

In [110]:
# Parameters
n = 2  # Number of qubits
m = 2  # Truncation parameter
delta = 0.1  # error << 1

# Generate density matrix
a_x = 1.0  # Coefficient for s_x s_x
h_x = 0.5  # Coefficient for s_z

rho_theta, rho_theta_delta = generate_rho_rho_delta(a_x, h_x, delta)

import pandas as pd
pd.DataFrame(rho_theta)

,0,1,2,3
0,0.353091+0.000000j,0.0-0.0j,0.0-0.0j,0.323455+0.351845j
1,0.000000+0.000000j,0.0+0.0j,0.0+0.0j,0.000000+0.000000j
2,0.000000+0.000000j,0.0+0.0j,0.0+0.0j,0.000000+0.000000j
3,0.323455-0.351845j,0.0-0.0j,0.0-0.0j,0.646909+0.000000j


In [111]:
# Sub/Super Quantum Fisher Information (SSQFI) bounds
def compute_ssqfi_bounds(rho, rho_delta):
    trace_rho_sigma = np.trace(np.dot(rho, rho_delta))
    trace_rho2 = np.trace(np.dot(rho, rho))
    trace_sigma2 = np.trace(np.dot(rho_delta, rho_delta))

    sub_fidelity = trace_rho_sigma - np.sqrt(
        2
        * (
            trace_rho_sigma**2
            - np.trace(np.dot(np.dot(rho, rho_delta), np.dot(rho, rho_delta)))
        )
    )
    super_fidelity = trace_rho_sigma + np.sqrt((1 - trace_rho2) * (1 - trace_sigma2))

    # Compute SSQFI bounds
    lower_ssqfi = 8 * (1 - super_fidelity) / (delta**2)
    upper_ssqfi = 8 * (1 - sub_fidelity) / (delta**2)
    return lower_ssqfi, upper_ssqfi

In [95]:
i, j = compute_ssqfi_bounds(rho_theta, rho_theta_delta)
i, j = np.real(i), np.real(j)
print(f"lower = {(i)},\nupper = {(j)},\ndiff = ({(j-i)})")

lower = 3.851993527484065,
upper = 3.852005472479902,
diff = (1.1944995836898897e-05)


In [104]:
def validate_density_matrix(rho):
    # Check Hermiticity
    if not np.allclose(rho, rho.conj().T, atol=1e-10):
        raise ValueError("Density matrix is not Hermitian.")

    # Check Trace is one
    if not np.isclose(np.trace(rho), 1.0, atol=1e-10):
        raise ValueError("Trace of density matrix is not 1.")

    # Check Positive Semi-Definiteness
    eigvals = np.linalg.eigvalsh(rho)
    if np.any(eigvals < -1e-10):  # Allow for small numerical errors
        raise ValueError("Density matrix has negative eigenvalues.")

    # Check if pure or not
    if np.isclose(np.trace(rho @ rho), 1.0, atol=1e-10):
        print("Density matrix is pure")
    else:
        print("Density matrix is mixed")
    
    print("and valid.")


In [105]:

validate_density_matrix(rho_theta)
validate_density_matrix(rho_theta_delta)


Density matrix is pure
and valid.
Density matrix is pure
and valid.


In [98]:
def compute_ssqfi_bounds_enforced(rho, rho_delta):
    # Validate inputs
    #validate_density_matrix(rho)
    #validate_density_matrix(rho_delta)

    # Compute traces
    trace_rho_sigma = np.real(np.trace(np.dot(rho, rho_delta)))
    trace_rho2 = np.real(np.trace(np.dot(rho, rho)))
    trace_sigma2 = np.real(np.trace(np.dot(rho_delta, rho_delta)))

    # Compute fidelities
    sub_fidelity = trace_rho_sigma - np.sqrt(
        max(0, 2 * (
            trace_rho_sigma**2
            - np.real(np.trace(np.dot(np.dot(rho, rho_delta), np.dot(rho, rho_delta))))
        ))
    )
    super_fidelity = trace_rho_sigma + np.sqrt(
        max(0, (1 - trace_rho2) * (1 - trace_sigma2))
    )

    # Compute SSQFI bounds
    lower_ssqfi = 8 * (1 - super_fidelity) / (delta**2)
    upper_ssqfi = 8 * (1 - sub_fidelity) / (delta**2)

    return lower_ssqfi, upper_ssqfi


In [99]:
i, j = compute_ssqfi_bounds_enforced(rho_theta, rho_theta_delta)
print(f"lower = {i},\nupper = {j},\ndiff = ({j-i})")

lower = 3.8519935274840646,
upper = 3.8520054484130197,
diff = (1.1920928955078125e-05)
